In [40]:
import json
import sys
from IPython.display import display, Markdown
sys.path.append("../src")
from agentix import AgentixConfig
from agentix.agent import agentix
from agentix.sessions import assemble_prompts

In [2]:
test_project_path = "/home/mpeters/starbucks/projects/retail-fraud-analytics/python"

In [6]:
config=AgentixConfig()
config.model="gpt"
config.system=["python_coder", "modifier_class_decorator","structured_response"]
config.tools=["cst"]
config.with_frontend=True
config.debug=False
config.user = [
    ("Create a plan to identify classes that implement both modif() and requires() "
    "methods so you can create a class decortator to describe the modifycation.  Identify tools to call"),
]


In [7]:
p = assemble_prompts(config, [], 4000)
print(json.dumps(p, indent=2).encode("utf8").decode("unicode_escape"))

{
  "model": "gpt",
  "messages": [
    {
      "role": "tool_calls",
      "content": "[TOOLS]
[
  [
    {
      "type": "function",
      "function": {
        "name": "module_files",
        "description": "Get all Python files in a module directory (recursive) whose path (relative",
        "parameters": {
          "properties": {
            "root_path": {
              "type": "str"
            },
            "module_prefix": {
              "type": "str"
            }
          }
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "cst_tree",
        "description": "Parse a Python file into a LibCST Module (Concrete Syntax Tree).",
        "parameters": {
          "properties": {
            "file_path": {
              "type": "str"
            }
          }
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "cst_modules",
        "description": "Parse multiple Python files into LibCST Modules.

In [8]:
results:dict = agentix(config)

Debug: args.session = agentix_session
Debug: Invoking manage_sessions
Managing session: agentix_session
Creating new session...
Debug: Calling summarize_user_prompt with args: AgentixConfig(debug=False, list_models=False, list_sessions=False, list_prompts=False, session='agentix_session', system=['python_coder', 'modifier_class_decorator', 'structured_response'], model='gpt-oss:latest', temperature=0.7, user=['Create a plan to identify classes that implement both modif() and requires() methods so you can create a class decortator to describe the modifycation.  Identify tools to call'], file_path=None, replace_file=False, serve=False, port=8000, with_frontend=True, tools=['cst'])
Session IdentifyOverrideableClassesAndTools created.
Debug: args.session = IdentifyOverrideableClassesAndTools
Debug: args = AgentixConfig(debug=False, list_models=False, list_sessions=False, list_prompts=False, session='IdentifyOverrideableClassesAndTools', system=['python_coder', 'modifier_class_decorator', '

In [44]:
response_raw = results["response"].replace("\n","").replace("\t","")

response = json.loads(response_raw)
for r in response:
    for k,v in r.items():
        match k:
            case "markdown":
                display(Markdown(v))
            case "next_steps":
                display(Markdown("### Next Steps\n"))
                for step in v:
                    display(f"-- {step}")
            case _:
                print(k)
                print(v)
                print("-----")


agent
Here is a concise plan and the tools you can use to discover all classes that define both `modify()` and `requires()` methods, so you can later apply a class decorator to describe their behavior.
-----


## Plan Overview

1. **Project Traversal** – Walk the repository to locate all `.py` files that could contain relevant classes.
2. **AST Parsing** – For each file, parse the code into an Abstract Syntax Tree (AST) to reliably locate class definitions and their method names.
3. **Method Matching** – Within each class node, check if both `modify` and `requires` are defined (case‑sensitive). Ignore nested classes or functions.
4. **Collect Metadata** – For matched classes, capture:
   * File path
   * Class name
   * Method signatures (e.g., parameters, return types)
   * Any existing decorators or docstrings
5. **Generate Decorator** – Using the collected metadata, programmatically insert or update the `etl_step` decorator with the required attributes.
6. **Apply Changes** – Write the modified source back to disk (or create a diff) while preserving original formatting and comments.

## Suggested Toolset

| Tool | Purpose | Example Usage |
|------|---------|---------------|
| `pathlib.Path` | File system traversal | `Path('src').rglob('*.py')` |
| `ast` | Parse Python source into AST | `ast.parse(source)` |
| `ast.NodeVisitor` | Walk the AST and collect nodes | subclass to find `ClassDef` nodes |
| `inspect` | For dynamic introspection if files can be imported | `inspect.getmembers(module, inspect.isclass)` |
| `importlib.util` | Load modules from file paths without executing side effects | `spec_from_file_location` |
| `black` / `autopep8` | Optional formatting after modifications |
| `git` | Commit changes or generate patches |
| `pydocstyle` | Ensure docstrings before inserting decorator |

## Quick Implementation Sketch

```python
import ast
from pathlib import Path

class ModifyRequiresVisitor(ast.NodeVisitor):
    def __init__(self):
        self.matches = []

    def visit_ClassDef(self, node):
        method_names = {n.name for n in node.body if isinstance(n, ast.FunctionDef)}
        if {'modify', 'requires'} <= method_names:
            self.matches.append((node.name, node.lineno))
        self.generic_visit(node)

def find_classes(root: Path):
    visitor = ModifyRequiresVisitor()
    for py_file in root.rglob('*.py'):
        source = py_file.read_text()
        tree = ast.parse(source, filename=str(py_file))
        visitor.visit(tree)
    return visitor.matches
```

You can then iterate over the matches to insert the `@etl_step` decorator via string manipulation or a dedicated AST transformer.


### Next Steps


'-- Run the above sketch against your repository to build a registry of candidate classes.'

'-- Extend the visitor to capture method signatures and docstrings for richer decorator metadata.'

'-- Write a separate AST transformer that inserts or updates the `etl_step` decorator based on the captured data.'

In [ ]:
config=AgentixConfig()
config.system=["python_coder", "modifier_class_decorator","structured_response"]
config.file = test_project_path
config.user = ("Create a plan to identify classes that implement both modif() and requires() "
               "methods so you can create a class decortator to describe the modifycation.")

In [ ]:
agentix(config)